# Politecnico Malvinas Argentinas- PDH: Preprocesamiento, Normalización y Técnicas Básicas

In [38]:
# Instalación de librerías necesarias (si es necesario en Colab)
!pip install nltk spacy scikit-learn thefuzz stop_words
!pip install fuzzywuzzy python-Levenshtein

import re
import nltk
import spacy
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sil46\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sil46\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
# Cargar modelo en español de spaCy
!python -m spacy download es_core_news_md
nlp = spacy.load("es_core_news_md")

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
      --------------------------------------- 0.8/42.3 MB 3.4 MB/s eta 0:00:13
     - -------------------------------------- 1.3/42.3 MB 3.9 MB/s eta 0:00:11
     -- ------------------------------------- 2.4/42.3 MB 3.7 MB/s eta 0:00:11
     -- ------------------------------------- 3.1/42.3 MB 3.8 MB/s eta 0:00:11
     --- ------------------------------------ 4.2/42.3 MB 4.0 MB/s eta 0:00:10
     ----- ---------------------------------- 5.8/42.3 MB 4.6 MB/s eta 0:00:08
     ------ --------------------------------- 7.1/42.3 MB 4.8 MB/s eta 0:00:08
     ------- -------------------------------- 8.1/42.3 MB 4.9 MB/s eta 0:00:07
     -------- ------------------------------- 9.4/42.3 MB 5.1 MB/s eta 0:00:07
     ---------- ----------------------------- 11.0/42.3 MB 5.3 MB/s eta 0:00:06
     ----------- ---------------------------- 12.3/42.3 MB 5.4 MB


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# 1. Preprocesamiento de texto

# Tokenizacion

Quizás una de las operaciones más básicas que podemos hacer es contar palabras. Imaginemos un caso extremadamente sencillo relacionado al aprendizaje automático. Pensemos en un clasificador de positividad / negatividad.

Podríamos usar una regla como:

si "bueno" está en el texto, clasificamos como positivo.

Claramente un enfoque tan sencillo va a ser propenso a muchos errores. Iremos viendo cómo aplicarlo y mejorarlo.

La operación de separar los textos en unidades básicas (o tokens) la llamamos tokenización.

In [41]:
doc2 = 'Usualmente, existe una relación costo-beneficio entre las distintas técnicas.'
doc2.split(' ')

['Usualmente,',
 'existe',
 'una',
 'relación',
 'costo-beneficio',
 'entre',
 'las',
 'distintas',
 'técnicas.']

Vemos que en castellano, el lenguaje natural (escrito) puede tener distintas fuentes de variaciones:

Mayúsculas
Tildes
Signos ortográficos
Errores de tipeo
Variaciones propias del lenguaje como la conjugación
Artefactos de la escritura informal como "holaaa"
¿Cómo abordarán esto nuestros algorítmos?

Una solución es la normalización: con acepción similar aunque distinta a la que usamos en estadística, va a significar transformar a los strings a representaciones iguales.

Podemos:

Sacar espacios
Pasar a minúsculas
Quitar tildes

In [42]:
%%capture
!pip install unidecode

In [43]:
import unidecode
[unidecode.unidecode(w.lower().strip()) for w in doc2.split(' ')]

['usualmente,',
 'existe',
 'una',
 'relacion',
 'costo-beneficio',
 'entre',
 'las',
 'distintas',
 'tecnicas.']

Otras normalizaciones más sofisticadas serían:
- Pasar de plural a singular
- Convertir el género de la palabra
- Conjugar los verbos en infinitivo

¡Es de esperarse que esto requiera métodos más complejos!

Vemos que con la tokenización anterior, todavía tenemos palabras con signos de puntuación pegados. Armar nuestro vocabulario es un paso clave para nuestros modelos de *Machine Learning*. 

In [44]:

text = "¡Hola! ¿Cómo estás? Estoy aprendiendo NLP con spaCy y NLTK. NLP es fascinante."

# Tokenización y Normalización
words = word_tokenize(text.lower())
print("Tokenización y Normalización:", words)




Tokenización y Normalización: ['¡hola', '!', '¿cómo', 'estás', '?', 'estoy', 'aprendiendo', 'nlp', 'con', 'spacy', 'y', 'nltk', '.', 'nlp', 'es', 'fascinante', '.']


In [45]:
# Expresiones Regulares
text_clean = re.sub(r'[^a-zA-Záéíóúüñ ]', '', text.lower())
print("Texto limpio con regex:", text_clean)



Texto limpio con regex: hola cómo estás estoy aprendiendo nlp con spacy y nltk nlp es fascinante


In [46]:
# Stop Words
stop_words = set(stopwords.words('spanish'))
filtered_words = [word for word in words if word not in stop_words]
print("Sin Stop Words:", filtered_words)

Sin Stop Words: ['¡hola', '!', '¿cómo', '?', 'aprendiendo', 'nlp', 'spacy', 'nltk', '.', 'nlp', 'fascinante', '.']


# 2. Técnicas de NLP con NLTK y spaCy

In [47]:
# Stemming con NLTK
stemmer = SnowballStemmer("spanish")
stemmed_words = [stemmer.stem(word) for word in filtered_words]
print("Stemming:", stemmed_words)


Stemming: ['¡hol', '!', '¿com', '?', 'aprend', 'nlp', 'spacy', 'nltk', '.', 'nlp', 'fascin', '.']


In [48]:
# Lemmatización con spaCy
doc = nlp(" ".join(filtered_words))
lemmas = [token.lemma_ for token in doc]
print("Lemmatización:", lemmas)

Lemmatización: ['¡', 'hola', '!', '¿', 'cómo', '?', 'aprender', 'nlp', 'spacy', 'nltk', '.', 'nlp', 'fascinante', '.']


In [49]:
# Distancias entre strings
print("Distancia fuzzy entre 'casa' y 'casas':", fuzz.ratio("casa", "casas"))
texts = ["El gato come pescado", "El perro ladra en la casa"]

Distancia fuzzy entre 'casa' y 'casas': 89


In [2]:
# segundo ejemplo %%capture
!pip install pyjarowinkler
from nltk.metrics import edit_distance
from pyjarowinkler import distance as jwdist


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Definimos una lista de tuplas de palabras
palabras = [("pero", "perro"),("pero", "pierdo"), ("nueve", "mueve"),  ("totalmente","diferentes"), ("pero", "basta")]

# Calculamos las metricas de distancia pasando cada tupla como argumentos a levdist() y get_jaro_distance()
for x,y in palabras:
    print(f"'{x}' vs '{y}':")
    print("Distancia Levenshtein ->", edit_distance(x,y))
    print("Similitud Jaro Winkler ->",jwdist.get_jaro_distance(x,y))
    print("-"*40)

'pero' vs 'perro':
Distancia Levenshtein -> 1
Similitud Jaro Winkler -> 0.07
----------------------------------------
'pero' vs 'pierdo':
Distancia Levenshtein -> 2
Similitud Jaro Winkler -> 0.11
----------------------------------------
'nueve' vs 'mueve':
Distancia Levenshtein -> 1
Similitud Jaro Winkler -> 0.13
----------------------------------------
'totalmente' vs 'diferentes':
Distancia Levenshtein -> 7
Similitud Jaro Winkler -> 0.48
----------------------------------------
'pero' vs 'basta':
Distancia Levenshtein -> 5
Similitud Jaro Winkler -> 1.0
----------------------------------------


# 3. Vectorización y Representación del Texto

In [50]:
texts = ["El gato come pescado", "El perro ladra en la casa"]

In [51]:
# Bolsa de Palabras (BoW)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())


,casa,come,el,en,gato,la,ladra,perro,pescado
0,0,1,1,0,1,0,0,0,1
1,1,0,1,1,0,1,1,1,0


In [52]:
# TF-IDF
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(texts)
pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names_out())

,casa,come,el,en,gato,la,ladra,perro,pescado
0,0.00000,0.534046,0.379978,0.00000,0.534046,0.00000,0.00000,0.00000,0.534046
1,0.42616,0.000000,0.303216,0.42616,0.000000,0.42616,0.42616,0.42616,0.000000


In [53]:
# N-gramas
vectorizer_ngram = CountVectorizer(ngram_range=(2,3), min_df=1)
X_ngram = vectorizer_ngram.fit_transform(texts)
pd.DataFrame(X_ngram.toarray(), columns=vectorizer_ngram.get_feature_names_out())


,come pescado,el gato,el gato come,el perro,el perro ladra,en la,en la casa,gato come,gato come pescado,la casa,ladra en,ladra en la,perro ladra,perro ladra en
0,1,1,1,0,0,0,0,1,1,0,0,0,0,0
1,0,0,0,1,1,1,1,0,0,1,1,1,1,1


# 4. Tareas Avanzadas con spaCy

In [54]:
sentence = "El presidente de Argentina visitó Buenos Aires."
doc = nlp(sentence)


In [55]:
# POS Tagging
print([(token.text, token.pos_) for token in doc])

[('El', 'DET'), ('presidente', 'NOUN'), ('de', 'ADP'), ('Argentina', 'PROPN'), ('visitó', 'VERB'), ('Buenos', 'PROPN'), ('Aires', 'PROPN'), ('.', 'PUNCT')]


In [56]:
# Named Entity Recognition (NER)
print([(ent.text, ent.label_) for ent in doc.ents])

[('Argentina', 'LOC'), ('Buenos Aires', 'LOC')]


In [57]:
# Dependencias sintácticas
displacy.render(doc, style='dep', jupyter=True)

NameError: name 'displacy' is not defined

In [34]:

# Limitaciones de Bolsa de Palabras
print("BoW no captura el significado de las palabras ni el orden de las mismas.")

BoW no captura el significado de las palabras ni el orden de las mismas.
